In [ ]:
import pandas as pd
import statistics
import datetime 

In [ ]:
# 計算報酬率
def sell_return(today_nv, buy_nv, today_date, buy_date): # 計算賣出報酬率
    hold_days = (today_date - buy_date).days
    ret = (today_nv - buy_nv)/ buy_nv

    return ret

In [ ]:
# import data #
#基金資料, 多 read 幾個sheet包成迴圈就可以一次測很多檔基金，可以把結果輸出成excel
df_fund = pd.read_excel("基金資料.xlsx") 
df_fund['年月日'] = pd.to_datetime(df_fund['年月日'])



In [ ]:
#低檔投資法回測 ：10% 加碼, 淨值若比10天前下跌10%,「隔天」就加碼
df_10_percent = pd.DataFrame(columns = ["買進日","買入價格","加碼"]) # 存回測結果
unit = 1 # 持有單位,假設創立之初先買一單位
start_index = 0 # 測試不同時間開始的報酬率
buy_nv = df_fund.loc[0,'淨值(元)'] # 假設成立當日就先買一單位
df_10_percent = df_10_percent.append({"買進日":df_fund.loc[0,'年月日'],
                                      "買入價格" : buy_nv, 
                                      "加碼": "x"}, ignore_index=True)

for i,item in df_fund.iterrows():
    if((item['年月日']-df_fund.loc[start_index,'年月日']).days >= 30): # 定期定額
        unit = unit + 1
        buy_nv = buy_nv + df_fund.loc[i,'淨值(元)']
        start_index = i
        df_10_percent = df_10_percent.append({"買進日": item['年月日'], 
                                              "買入價格" : item['淨值(元)'], 
                                              "加碼": "x"}, ignore_index=True)
    
    if(i>=10):
        print(i,":", (item['淨值(元)'] - df_fund.loc[i-10,'淨值(元)'])*100/df_fund.loc[i-10,'淨值(元)'])
        if( item['淨值(元)'] <= 0.9*(df_fund.loc[i-10,'淨值(元)']) ): # 加碼
            buy_date = df_fund.loc[i+1,'年月日']
            unit = unit + 1
            buy_nv = buy_nv + df_fund.loc[i+1,'淨值(元)']
            df_10_percent = df_10_percent.append({"買進日":buy_date, 
                                                  "買入價格" : df_fund.loc[i+1,'淨值(元)'], 
                                                  "加碼": "v"}, ignore_index=True)
print("end loop")
print(unit)

profit_ret = sell_return(unit*df_fund.loc[len(df_fund)-1,'淨值(元)'], buy_nv,
                         df_fund.loc[len(df_fund)-1,'年月日'], df_10_percent.loc[0,'買進日'])
    
#pd.set_option('display.max_rows', None)
df_10_percent